In [2]:
import sqlite3
import requests
import tweepy
import os
from dotenv import load_dotenv
import tempfile
from datetime import datetime
import pytz

# Charger les variables d'environnement
load_dotenv()

# Configuration
GITHUB_REPO = 'steampage-creation-date'
DB_FILE_PATH = 'steam_games.db'
TIMESTAMP_FILE = 'timestamp_last_tweet.txt'
PARIS_TZ = pytz.timezone('Europe/Paris')

def read_last_timestamp():
    try:
        with open(TIMESTAMP_FILE, 'r') as f:
            return int(f.read().strip())
    except FileNotFoundError:
        return 0

def write_last_timestamp(timestamp):
    with open(TIMESTAMP_FILE, 'w') as f:
        f.write(str(timestamp))

def download_db(url, local_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    print(f"Échec du téléchargement de la base de données. Code de statut: {response.status_code}")
    return False

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

def check_new_entries(conn, last_timestamp):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT steam_game_id, first_seen
        FROM games 
        WHERE first_seen > ?
        ORDER BY first_seen ASC
    """, (last_timestamp,))
    return cursor.fetchall()

def get_game_details(steam_game_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={steam_game_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data[str(steam_game_id)]['success']:
            return data[str(steam_game_id)]['data']
    return None

def filter_game(game_data):
    return game_data['type'] == 'game' and not game_data.get('dlc', False)

def get_twitter_api():
    auth = tweepy.OAuthHandler(
        os.getenv('TWITTER_CONSUMER_KEY'),
        os.getenv('TWITTER_CONSUMER_SECRET')
    )
    auth.set_access_token(
        os.getenv('TWITTER_ACCESS_TOKEN'),
        os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
    )
    return tweepy.API(auth)

def get_twitter_client():
    client = tweepy.Client(
        consumer_key=os.getenv('TWITTER_CONSUMER_KEY'),
        consumer_secret=os.getenv('TWITTER_CONSUMER_SECRET'),
        access_token=os.getenv('TWITTER_ACCESS_TOKEN'),
        access_token_secret=os.getenv('TWITTER_ACCESS_TOKEN_SECRET')
    )
    return client

def download_image(url):
    response = requests.get(url)
    if response.status_code == 200:
        with tempfile.NamedTemporaryFile(delete=False, suffix=".jpg") as temp_file:
            temp_file.write(response.content)
            return temp_file.name
    return None

def format_tweet_message(game_data):
    try:
        name = game_data['name']
        genres = ", ".join([genre['description'] for genre in game_data.get('genres', [])])
        developers = ", ".join(game_data.get('developers', []))
        description = game_data.get('short_description', '')
        app_id = game_data['steam_appid']
        
        # Construire le lien Steam
        steam_link = f"https://store.steampowered.com/app/{app_id}"
        
        # Construire le tweet avec le lien
        tweet = f"Name: {name}. {genres}. Dev: {developers}. Desc: \"{description}\". Link: {steam_link}"
        
        # Vérifier et ajuster la longueur du tweet si nécessaire
        if len(tweet) > 280:
            available_space = 280 - len(f"Name: {name}. {genres}. Dev: {developers}. Desc: \"...\". Link: {steam_link}")
            truncated_description = description[:available_space] + "..."
            tweet = f"Name: {name}. {genres}. Dev: {developers}. Desc: \"{truncated_description}\". Link: {steam_link}"
        
        return tweet
    except KeyError as e:
        print(f"Erreur lors du formatage du tweet: Clé manquante - {e}")
        return None
    except Exception as e:
        print(f"Erreur inattendue lors du formatage du tweet: {e}")
        return None

def send_tweet(message, image_url=None):
    api = get_twitter_api()
    client = get_twitter_client()
    try:
        media_ids = None
        if image_url:
            image_path = download_image(image_url)
            if image_path:
                uploaded_media = api.media_upload(filename=image_path)
                media_ids = [uploaded_media.media_id]
                os.unlink(image_path)  # Supprimer le fichier temporaire

        # Créer le tweet avec l'image si disponible
        response = client.create_tweet(text=message, media_ids=media_ids)
        return response.data['id']
    except Exception as e:
        print(f"Erreur lors de la création du tweet: {e}")
        return None

def main():
    db_url = f"https://raw.githubusercontent.com/{os.getenv('GITHUB_USERNAME')}/{GITHUB_REPO}/main/{DB_FILE_PATH}"
    last_timestamp = read_last_timestamp()
    new_last_timestamp = last_timestamp

    with tempfile.NamedTemporaryFile(delete=False, suffix='.db') as temp_db:
        if download_db(db_url, temp_db.name):
            conn = connect_to_db(temp_db.name)
            new_entries = check_new_entries(conn, last_timestamp)
            
            for steam_game_id, first_seen in new_entries:
                print(f"Nouveau jeu trouvé : Steam ID: {steam_game_id}, First Seen: {first_seen}")
                game_data = get_game_details(steam_game_id)
                if game_data and filter_game(game_data):
                    message = format_tweet_message(game_data)
                    if message:
                        image_url = game_data.get('header_image')
                        tweet_id = send_tweet(message, image_url)
                        if tweet_id:
                            print(f"Tweet publié pour {game_data['name']} (ID: {tweet_id})")
                            new_last_timestamp = max(new_last_timestamp, first_seen)
                        else:
                            print(f"Échec de la publication du tweet pour {game_data['name']}")
                    else:
                        print(f"Échec du formatage du tweet pour {game_data['name']}")
                else:
                    print(f"Le jeu avec Steam ID {steam_game_id} ne répond pas aux critères de tweet ou les détails n'ont pas pu être récupérés.")
            
            conn.close()
        else:
            print("Échec du téléchargement de la base de données")
    
    os.unlink(temp_db.name)  # Supprime le fichier temporaire

    if new_last_timestamp > last_timestamp:
        write_last_timestamp(new_last_timestamp)
        print(f"Timestamp mis à jour : {new_last_timestamp}")

if __name__ == "__main__":
    main()

In [3]:
import sqlite3
import requests
import os
from dotenv import load_dotenv
import tempfile
from datetime import datetime
import pytz

# Charger les variables d'environnement
load_dotenv()

# Configuration
GITHUB_REPO = 'steampage-creation-date'
DB_FILE_PATH = 'steam_games.db'
TIMESTAMP_FILE = 'timestamp_last_tweet.txt'
PREVIEW_FILE = 'tweet_previews.txt'
PARIS_TZ = pytz.timezone('Europe/Paris')

def read_last_timestamp():
    try:
        with open(TIMESTAMP_FILE, 'r') as f:
            return int(f.read().strip())
    except FileNotFoundError:
        return 0

def download_db(url, local_path):
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(local_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    print(f"Échec du téléchargement de la base de données. Code de statut: {response.status_code}")
    return False

def connect_to_db(db_path):
    return sqlite3.connect(db_path)

def check_new_entries(conn, last_timestamp):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT steam_game_id, first_seen
        FROM games 
        WHERE first_seen > ?
        ORDER BY first_seen ASC
    """, (last_timestamp,))
    return cursor.fetchall()

def get_game_details(steam_game_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={steam_game_id}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data[str(steam_game_id)]['success']:
            return data[str(steam_game_id)]['data']
    return None

def filter_game(game_data):
    return game_data['type'] == 'game' and not game_data.get('dlc', False)

def format_tweet_message(game_data):
    try:
        name = game_data['name']
        genres = ", ".join([genre['description'] for genre in game_data.get('genres', [])])
        developers = ", ".join(game_data.get('developers', []))
        description = game_data.get('short_description', '')
        app_id = game_data['steam_appid']
        
        # Construire le lien Steam
        steam_link = f"https://store.steampowered.com/app/{app_id}"
        
        # Construire le tweet avec le lien
        tweet = f"Name: {name}. {genres}. Dev: {developers}. Desc: \"{description}\". Link: {steam_link}"
        
        # Vérifier et ajuster la longueur du tweet si nécessaire
        if len(tweet) > 280:
            available_space = 280 - len(f"Name: {name}. {genres}. Dev: {developers}. Desc: \"...\". Link: {steam_link}")
            truncated_description = description[:available_space] + "..."
            tweet = f"Name: {name}. {genres}. Dev: {developers}. Desc: \"{truncated_description}\". Link: {steam_link}"
        
        return tweet
    except KeyError as e:
        print(f"Erreur lors du formatage du tweet: Clé manquante - {e}")
        return None
    except Exception as e:
        print(f"Erreur inattendue lors du formatage du tweet: {e}")
        return None

def write_tweet_previews(tweets):
    with open(PREVIEW_FILE, 'w', encoding='utf-8') as f:
        for tweet in tweets:
            f.write(f"{tweet}\n\n")  # Deux sauts de ligne pour séparer les tweets

def main():
    db_url = f"https://raw.githubusercontent.com/{os.getenv('GITHUB_USERNAME')}/{GITHUB_REPO}/main/{DB_FILE_PATH}"
    last_timestamp = read_last_timestamp()
    tweets_to_preview = []

    with tempfile.NamedTemporaryFile(delete=False, suffix='.db') as temp_db:
        if download_db(db_url, temp_db.name):
            conn = connect_to_db(temp_db.name)
            new_entries = check_new_entries(conn, last_timestamp)
            
            for steam_game_id, first_seen in new_entries:
                print(f"Nouveau jeu trouvé : Steam ID: {steam_game_id}, First Seen: {first_seen}")
                game_data = get_game_details(steam_game_id)
                if game_data and filter_game(game_data):
                    message = format_tweet_message(game_data)
                    if message:
                        tweets_to_preview.append(message)
                        print(f"Tweet formaté pour {game_data['name']}")
                    else:
                        print(f"Échec du formatage du tweet pour {game_data['name']}")
                else:
                    print(f"Le jeu avec Steam ID {steam_game_id} ne répond pas aux critères de tweet ou les détails n'ont pas pu être récupérés.")
            
            conn.close()
        else:
            print("Échec du téléchargement de la base de données")
    
    os.unlink(temp_db.name)  # Supprime le fichier temporaire

    if tweets_to_preview:
        write_tweet_previews(tweets_to_preview)
        print(f"{len(tweets_to_preview)} tweets formatés ont été écrits dans {PREVIEW_FILE}")
    else:
        print("Aucun nouveau tweet à prévisualiser.")

if __name__ == "__main__":
    main()